In [1]:
import numpy as np
import customDataset
from dotenv import load_dotenv
import os
import torch
import random
from torch.utils.data import DataLoader
import lfcm
import torch.nn as nn 
from pylab import zeros
import torch.backends.cudnn as cudnn
import trainingFunctionsLFCM as t

load_dotenv(override=True)

True

In [2]:
root_dir = os.getenv('ROOT_PATH')
split_train = 'train.csv'
split_val = 'valid.csv'
batch_size = 24 # 32
workers = 4
gpu = 0
gpus = [0]
weights = [0.33209667, 0.66790333] 
class_weights = torch.FloatTensor(weights).cuda()
optimizer_name = 'ADAM'
epochs = 32 # 301
start_epoch = 0
ImgSize = 299
print_freq = 25

In [3]:
model = lfcm.LFCM(gpu=gpu)

criterion = nn.CrossEntropyLoss(weight=class_weights).cuda(gpu)

Inception v3 pretraining: Hardcoded strict=False, Might be omiting layers


In [4]:
if optimizer_name == 'ADAM':
    print("Using ADAM optimizer")
    lr = 1e-6
    cnn_lr = 1e-7

if optimizer_name == 'ADAM':
    print("Using ADAM optimizer with: CNN lr: " + str(cnn_lr) + " , mm_lr: " + str(lr) )
    optimizer = torch.optim.Adam([
                    {'params': model.mm.parameters()},
                    {'params': model.cnn.parameters(), 'lr': cnn_lr}],
                                lr = lr)

# print(optimizer)

Using ADAM optimizer
Using ADAM optimizer with: CNN lr: 1e-07 , mm_lr: 1e-06


In [5]:
model = torch.nn.DataParallel(model, device_ids=gpus).cuda(gpu)
cudnn.benchmark = True

In [6]:
train_dataset = customDataset.CustomDatasetLFCM(root_dir, split_train, Rescale=0, RandomCrop=299, Mirror=True, embed_dir='train_embeddings')

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=workers, pin_memory=True)

In [7]:
len(train_dataset)

9434

In [8]:
# for i in range(1000):
#     if train_dataset[i][3][0] != 0:
#         print(f"{i}", train_dataset[i][3])

In [9]:
val_dataset = customDataset.CustomDatasetLFCM(
    root_dir, split_val, Rescale=ImgSize,RandomCrop=0,Mirror=False, embed_dir='valid_embeddings')

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=True, num_workers=workers, pin_memory=True)

In [10]:
# for i in range(1000):
#     if val_dataset[i][2][] == 0:
#         print(f"{i}", val_dataset[i][2])

In [11]:
# for i in range(len(val_dataset)):
#     if val_dataset[i][3][0] == 0:
#         print(val_dataset[i][3])
#         break

# print(val_dataset[45][3])

In [12]:
plot_data = {}
plot_data['train_loss'] = zeros(epochs)
plot_data['train_acc'] = zeros(epochs)
plot_data['train_acc_racist'] = zeros(epochs)
plot_data['train_acc_notRacist'] = zeros(epochs)
plot_data['train_acc_avg'] = zeros(epochs)
plot_data['val_loss'] = zeros(epochs)
plot_data['val_acc'] = zeros(epochs)
plot_data['val_acc_racist'] = zeros(epochs)
plot_data['val_acc_notRacist'] = zeros(epochs)
plot_data['val_acc_avg'] = zeros(epochs)
plot_data['epoch'] = 0

In [13]:
is_best = True
val_acc_avg_best = 0
epoch_best = -1
epoch_loss_best = 0
model_name = 'lfcm_v2_b16_toy'
best_loss = 0
print(batch_size)
for epoch in range(start_epoch, epochs):
    plot_data['epoch'] = epoch

    print('Epoch:', epoch)
    t.train(train_loader, model, criterion, optimizer, epoch, print_freq, plot_data, gpu)

    plot_data = t.validate(val_loader, model, criterion, print_freq, plot_data, gpu)

    if (plot_data['val_acc_avg'][epoch]) > val_acc_avg_best:
        t.save_checkpoint(model, is_best, filename=f'checkpoints/{model_name}')
        epoch_best = epoch
        val_acc_avg_best = plot_data['val_acc_avg'][epoch]

    # if best_loss == 0:
    #     print('loss:', plot_data['val_loss'][epoch])
    #     t.save_checkpoint(model, is_best, filename=f'checkpoints/val_loss_{model_name}')
    #     epoch_loss_best = epoch
    #     best_loss = plot_data['val_loss'][epoch]

    # if (plot_data['val_loss'][epoch]) < best_loss:
    #     print('loss:', plot_data['val_loss'][epoch])
    #     t.save_checkpoint(model, is_best, filename=f'checkpoints/val_loss_{model_name}')
    #     epoch_loss_best = epoch
    #     best_loss = plot_data['val_loss'][epoch]
    
print(f'best model avg acc epoch: {model_name} epoch: {epoch_best} batch_size: {batch_size}')
# print(f'best model loss epoch: {model_name} epoch: {epoch_best} batch_size: {batch_size}')

24
Epoch: 0


TRAIN:: Acc: 45.230018615722656Acc Avg: 49.7594928781104 Racist Acc: 63.50174805760985 - Not Racist Acc: 36.017237698610856
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image size: 24
image 

Traceback (most recent call last):
  File "c:\Users\John Glen\Desktop\Simplify\PUP\SY-2023-2024\tool\lfcm-racism\venv\Lib\site-packages\IPython\core\interactiveshell.py", line 3548, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\John Glen\AppData\Local\Temp\ipykernel_14256\3820799200.py", line 12, in <module>
    t.train(train_loader, model, criterion, optimizer, epoch, print_freq, plot_data, gpu)
  File "c:\Users\John Glen\Desktop\Simplify\PUP\SY-2023-2024\tool\lfcm-racism\src\training\trainingFunctionsLFCM.py", line 41, in train
    cur_acc_racist, cur_acc_notRacist = accuracy_per_class(output.data, target.long().cuda(gpu))
                                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\John Glen\Desktop\Simplify\PUP\SY-2023-2024\tool\lfcm-racism\src\training\trainingFunctionsLFCM.py", line -1, in accuracy_per_class
KeyboardInterrupt

During handling of the above exception, another exception occurre